In [1]:
!pip3 install wikipedia
!pip3 install networkx

  Using cached networkx-2.5-py3-none-any.whl (1.6 MB)


In [2]:
import yaml
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import gensim
from gensim import corpora
from os import listdir
from os.path import isfile, join
import wikipedia
from bs4 import BeautifulSoup
import requests
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import operator
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/douglas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# LDA


<b> Atividades </b>

1) Faça download dos arquivos no link: https://drive.google.com/open?id=1PgGJ2OBFSKnr3O0RIgO2EvgbLh_avRJN

2) Carregue os quatro documentos. Para carregar um arquivo ``yml``, siga o exemplo abaixo:
```python
with open('data/chats/file.yml') as file:
    chat = yaml.load(file)
```
<b> Dica: adicione os 4 documentos em um único dicionário </b>

In [6]:
arquivos = ["linguistic_knowledge", "money", "proverbs", "games"]

In [22]:
todos_chats = {}
for arquivo in arquivos:
    with open(arquivo+'.yml') as file:
        conversas = yaml.load(file)
        todos_chats[conversas["categories"][0]] = conversas["conversations"]

<ipython-input-22-5cd51610a7d1>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  conversas = yaml.load(file)


In [23]:
todos_chats.keys()

dict_keys(['conhecimento linguístico', 'money', 'proverbs', 'games'])

<b> Atividade </b>

A variável ``chat`` é um dicionário, que possui a chave ``conversations``, que contém várias listas. Dentro de cada lista, existe um par de frases, uma escrita por um humano e outra escrita por um robô. Neste exercício, só iremos considerar a frase do humano, ou seja, a primeira posição do par nas listas.
Exemplo:

```python
chat = {'categories': ['compliment'],
        'conversations': [['você é linda', 'eu sei'],
                          ['adoro você', 'também'],
                          ['gosto da sua sinceridade', 'que bom']]}
```
No exemplo acima, iremos considerar apenas as frases: <b> 'você é linda', 'adoro você' e 'gosto da sua sinceridade' </b>.

Também iremos representar os chats por uma única lista. Ou seja, as listas de frases selecionadas no exemplo anterior se tornarão uma única lista: <b> ['você é linda adoro você e gosto da sua sinceridade'] </b>



3) Conforme explicado anteriormente, faça as alterações necessárias no dado. Primeiro, considere apenas a frase da primeira posição, depois una todas as listas de frases em uma única lista.

In [24]:
for k in todos_chats.keys():
    todos_chats[k] = ' '.join([chat[0] for chat in todos_chats[k]])

In [25]:
todos_chats

{'conhecimento linguístico': 'o que é linguística? o que é um verbo? estuda quem foi Saussure? O que é uma Gramática O que é um substantivo? quem é Noam Chomsky? o que é um sujeito? Quem é Marcos Bagno? Quais os livros que todo estudante de Letras deve ter? Quais as áreas da Linguística? Quando uso o porque, o por que, o porquê e o porquê? Pode explicar o uso dos porquês? Qual a forma correta, concerteza ou com certeza? O que são as figuras de linguagem? o que é um pronome? o que é a ANPOLL? Quando será realizado o próximo SIC? Quais os pronomes pessoais do caso reto? Quem é o autor de Análise de Textos de Comunicação? O que é um advérbio? O que é a langue? O que é a parole O que é letramento? O que é a alfabetização?',
 'money': 'Você é pago mercado de ações mercado de ações mercado de ações mercado de ações mercado de ações mercado de ações mercado de ações mercado de ações taxa de juros o que é um dólar o que é dinheiro o que é o mercado de ações o que é o mercado de ações o que é o

<b> Atividade </b>

4) Faça uma limpeza nos dados carregados e processados do passo anterior. Aplique:
- tokenização
- lower
- remoção stopwords
- remoção pontuação

Ao final você deve obter uma lista com todos os documentos e em cada documento as palavras tokenizadas, exemplo:

```python
[['oi', 'tudo', 'bem'],
 ['mario', 'game', 'jogo']]
```

In [26]:
def pre_processamento_texto(corpus):

    #tokenizacao
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #capitalizacao
    corpus_alt = [t.lower() for t in corpus_alt]
    #remover stopwords
    stopwords_ = stopwords.words("portuguese")
    corpus_alt = [t for t in corpus_alt if t not in stopwords_]
    #remover numero
    corpus_alt = [re.sub(r"\d","",t) for t in corpus_alt]
    #remover pontuacoes
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

    return corpus_alt

In [34]:
todos_chats_limpo = []
for k in todos_chats.keys():
    todos_chats_limpo.append(pre_processamento_texto(todos_chats[k]))

<b> Atividade </b>

5) Crie um dicionário com nosso corpus, de forma que cada termo único (valor) possua um índice (chave). 

Dica: 
- utilize o ```gensim.corpora.Dictionary ``` para criar este dicionário. 
- utilize o texto pré-processado

In [35]:
dicionario = corpora.Dictionary(todos_chats_limpo)

In [36]:
len(dicionario)

244

<b> Atividade </b>

6) Converta os documentos em índices, utilizando o método ```doc2bow``` em cada documento.

In [37]:
todos_chats_limpo

[['linguística',
  'verbo',
  'estuda',
  'saussure',
  'gramática',
  'substantivo',
  'noam',
  'chomsky',
  'sujeito',
  'marcos',
  'bagno',
  'quais',
  'livros',
  'todo',
  'estudante',
  'letras',
  'deve',
  'ter',
  'quais',
  'áreas',
  'linguística',
  'uso',
  'porque',
  'porquê',
  'porquê',
  'pode',
  'explicar',
  'uso',
  'porquês',
  'forma',
  'correta',
  'concerteza',
  'certeza',
  'figuras',
  'linguagem',
  'pronome',
  'anpoll',
  'realizado',
  'próximo',
  'sic',
  'quais',
  'pronomes',
  'pessoais',
  'caso',
  'reto',
  'autor',
  'análise',
  'textos',
  'comunicação',
  'advérbio',
  'langue',
  'parole',
  'letramento',
  'alfabetização'],
 ['pago',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'taxa',
  'juros',
  'dólar',
  'dinheiro',
  'mercado',
  'ações',
  'mercado',
  'ações',
  'mercado',
  'ações

In [38]:
doc_bow = [dicionario.doc2bow(chat) for chat in todos_chats_limpo]

In [40]:
len(doc_bow)

4

<b> Atividade </b>

7) Aplique o modelo LDA do Gensim nos dados processados acima. O modelo recebe a matriz de índices gerada anteriormente.
Abaixo, apresentamos um exemplo de como aplicar o modelo.


```python
model = gensim.models.ldamodel.LdaModel(doc_term_matrix, num_topics=4, id2word = dictionary, passes=1000, random_state=123)
#para mostrar os tópicos:
model.show_topics()
```

Fique a vontade para alterar os parâmetros e fazer testes.

In [41]:
model = gensim.models.ldamodel.LdaModel(doc_bow, num_topics=4, id2word = dicionario, passes=1000, random_state=123)

In [42]:
model.show_topics()

[(0,
  '0.099*"ações" + 0.099*"mercado" + 0.064*"quanto" + 0.046*"dinheiro" + 0.029*"economia" + 0.029*"ganha" + 0.020*"dólar" + 0.011*"juros" + 0.011*"pago" + 0.011*"real"'),
 (1,
  '0.028*"quais" + 0.020*"porquê" + 0.020*"uso" + 0.020*"linguística" + 0.011*"pronomes" + 0.011*"pronome" + 0.011*"porquês" + 0.011*"pode" + 0.011*"pessoais" + 0.011*"porque"'),
 (2,
  '0.015*"ter" + 0.015*"vou" + 0.010*"comprar" + 0.010*"trabalho" + 0.010*"quer" + 0.010*"conseguiu" + 0.010*"tudo" + 0.010*"faça" + 0.010*"pai" + 0.010*"·"'),
 (3,
  '0.060*"acha" + 0.031*"chamado" + 0.031*"programa" + 0.031*"conhece" + 0.021*"console" + 0.021*"sonic" + 0.021*"mario" + 0.012*"consoles" + 0.012*"devo" + 0.012*"creed"')]

<b> Atividade </b>

8) Para responder esta questão utilize os parâmetros sugeridos na questão 7. Analise os grupos gerados anteriormente e responda:
- a) Os grupos são bem dividos? Caracterize-os.
- b) Dê um nome para cada grupo de tópicos.

a) os grupos são bem dividos, mas o grupo que seria relativo à "provérbios" ficou caracterizado com algo parecido como "aquisições" ou "posses".

d) tópico 1: finanças/mercado; tópico 2: linguagens; tópico 3: aquisições; tópico 4: jogos 

# Sumarização

Aqui iremos realizar uma sumarização de notíticas com técnicas que vimos nas aulas anteriores.

<b> Atividade </b>

9) Faça download do texto da seguinte notícia: 

``https://g1.globo.com/sp/sao-paulo/noticia/2020/03/03/cai-para-130-numero-de-casos-suspeitos-de-coronavirus-no-estado-de-sao-paulo.ghtml``

Dica: para fazer download, consulte a Aula Prática 4.

In [44]:
def match_class(target):                                                        
    def do_match(tag):                                                          
        classes = tag.get('class', [])                                          
        return all(c in classes for c in target)                                
    return do_match 

def get_text_url(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    #remove marcações de scripts e style
    texto = soup.find_all(match_class(["content-text__container"]))
    all_text = ""
    for t in texto:
        all_text += t.get_text()
    return all_text

In [47]:
noticia = get_text_url("https://g1.globo.com/sp/sao-paulo/noticia/2020/03/03/cai-para-130-numero-de-casos-suspeitos-de-coronavirus-no-estado-de-sao-paulo.ghtml")

<b> Atividade </b>

10) Imprima o texto coletado.

In [48]:
noticia

' A Secretaria Estadual de Saúde de São Paulo divulgou nesta terça-feira (3) que caiu o número de pacientes com suspeita de coronavírus no estado. Enquanto na segunda-feira (2) eram 163 casos suspeitos, nesta terça são 130.  Foram descartados, ao todo, 104 casos no estado, destes, 56 foram nas últimas 24 horas. Entretanto, segundo informou a pasta, entre segunda (2) e terça (3), 23 novos surgiram.  Permanecem confirmados apenas 2 casos da doença no país, ambos no estado de São Paulo. Os pacientes estão em quarentena domiciliar. Ambos estiveram em viagem na Itália.  "Os dois confirmados estão evoluindo bem clinicamente, mas ainda têm alguns sintomas que fazem com que eles continuem em isolamento domiciliar", salientou Paulo Menezes, coordenador do comitê de operações emergenciais (COE) da Secretaria Estadual de Saúde.  O número de pessoas que tiveram contato com o segundo caso confirmado não foi divulgado pela secretaria. No primeiro caso confirmado, eram 34 pessoas, entre passageiros d

<b> Atividade </b>

11) Faça a tokenização das sentenças.

In [51]:
noticia_sents = sent_tokenize(noticia)

In [52]:
noticia_sents

[' A Secretaria Estadual de Saúde de São Paulo divulgou nesta terça-feira (3) que caiu o número de pacientes com suspeita de coronavírus no estado.',
 'Enquanto na segunda-feira (2) eram 163 casos suspeitos, nesta terça são 130.',
 'Foram descartados, ao todo, 104 casos no estado, destes, 56 foram nas últimas 24 horas.',
 'Entretanto, segundo informou a pasta, entre segunda (2) e terça (3), 23 novos surgiram.',
 'Permanecem confirmados apenas 2 casos da doença no país, ambos no estado de São Paulo.',
 'Os pacientes estão em quarentena domiciliar.',
 'Ambos estiveram em viagem na Itália.',
 '"Os dois confirmados estão evoluindo bem clinicamente, mas ainda têm alguns sintomas que fazem com que eles continuem em isolamento domiciliar", salientou Paulo Menezes, coordenador do comitê de operações emergenciais (COE) da Secretaria Estadual de Saúde.',
 'O número de pessoas que tiveram contato com o segundo caso confirmado não foi divulgado pela secretaria.',
 'No primeiro caso confirmado, era

<b> Atividade </b>

12) Aplique o pré-processamento nos dados:

- tokenização
- lower
- remoção stopwords
- remoção pontuação

In [53]:
def pre_processamento_texto(corpus):

    #tokenizacao
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #capitalizacao
    corpus_alt = [t.lower() for t in corpus_alt]
    #remover stopwords
    stopwords_ = stopwords.words("portuguese")
    corpus_alt = [t for t in corpus_alt if t not in stopwords_]
    #remover numero
    corpus_alt = [re.sub(r"\d","",t) for t in corpus_alt]
    #remover pontuacoes
    corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

    return corpus_alt

In [55]:
noticias_processado = [pre_processamento_texto(sent) for sent in noticia_sents]

In [56]:
noticias_processado

[['secretaria',
  'estadual',
  'saúde',
  'paulo',
  'divulgou',
  'nesta',
  'terça',
  'feira',
  'caiu',
  'número',
  'pacientes',
  'suspeita',
  'coronavírus',
  'estado'],
 ['enquanto', 'segunda', 'feira', 'casos', 'suspeitos', 'nesta', 'terça'],
 ['descartados', 'todo', 'casos', 'estado', 'destes', 'últimas', 'horas'],
 ['entretanto',
  'segundo',
  'informou',
  'pasta',
  'segunda',
  'terça',
  'novos',
  'surgiram'],
 ['permanecem',
  'confirmados',
  'apenas',
  'casos',
  'doença',
  'país',
  'ambos',
  'estado',
  'paulo'],
 ['pacientes', 'quarentena', 'domiciliar'],
 ['ambos', 'viagem', 'itália'],
 ['dois',
  'confirmados',
  'evoluindo',
  'bem',
  'clinicamente',
  'ainda',
  'têm',
  'alguns',
  'sintomas',
  'fazem',
  'continuem',
  'isolamento',
  'domiciliar',
  'salientou',
  'paulo',
  'menezes',
  'coordenador',
  'comitê',
  'operações',
  'emergenciais',
  'coe',
  'secretaria',
  'estadual',
  'saúde'],
 ['número',
  'pessoas',
  'contato',
  'segundo',
 

<b> Atividade </b>

13) Agora iremos implementar a sumarização. Para sumarizar você precisará implementar um método para cada uma das orientações abaixo:

13.1) Faça um método que calcule a similaridade entre duas sentenças utilizando a similaridade do conseno. 

Dica: faça a tranformação aplicando Bag of Words. Consulte a Aula 3.

In [79]:
palavras_unicas = (set(noticias_processado[0] + noticias_processado[1]))

In [80]:
model = CountVectorizer(binary=True)

In [81]:
model.fit(palavras_unicas)

CountVectorizer(binary=True)

In [82]:
sent1 = noticias_processado[0]
sent2 = noticias_processado[1]

In [83]:
vetor_s1 = model.transform([' '.join(sent1)]).todense()
vetor_s2 = model.transform([' '.join(sent2)]).todense()   

In [84]:
(model.get_feature_names())

['caiu',
 'casos',
 'coronavírus',
 'divulgou',
 'enquanto',
 'estado',
 'estadual',
 'feira',
 'nesta',
 'número',
 'pacientes',
 'paulo',
 'saúde',
 'secretaria',
 'segunda',
 'suspeita',
 'suspeitos',
 'terça']

In [85]:
cosine_similarity(vetor_s1, vetor_s2).reshape(-1)[0]
#cosine_similarity(vetor_s1, vetor_s2).reshape[0][0]

0.3030457633656632

In [197]:
def similaridade(s1, s2):
    palavras_unicas = (set(s1 + s2))
    model = CountVectorizer(binary=True)
    model.fit(palavras_unicas)
    vetor_s1 = model.transform([' '.join(s1)]).todense()
    vetor_s2 = model.transform([' '.join(s2)]).todense()    
    return cosine_similarity(vetor_s1, vetor_s2).reshape(-1)[0]

In [198]:
similaridade(noticias_processado[0], noticias_processado[0])

0.9999999999999999

13.2) Faça um método para construir uma matriz de similaridades entre as sentenças. 

Exemplo, seu texto tem 10 sentenças, você deve construir uma matriz (10,10). A posição (0,1) da matriz corresponderá a similaridade do cosseno entre a sentença 0 e a sentença 1.

In [199]:
qtde_sents = len(noticia_sents)

In [200]:
vetor_similaridades = np.zeros((len(noticia_sents) , len(noticia_sents)))

In [201]:
similaridade(noticias_processado[0], noticias_processado[1])

0.3030457633656632

In [202]:
for linha in range(0, qtde_sents):
    for coluna in range(0, qtde_sents):
        if linha == coluna:
            continue
        vetor_similaridades[linha][coluna] = similaridade(noticias_processado[linha], noticias_processado[coluna])
        

In [233]:
def matriz_similarides(sents):
    qtde_sents = len(noticia_sents)
    vetor_similaridades = np.zeros((len(noticia_sents) , len(noticia_sents)))
    for linha in range(0, qtde_sents):
        for coluna in range(0, qtde_sents):
            if linha == coluna:
                continue
            vetor_similaridades[linha][coluna] = similaridade(noticias_processado[linha], noticias_processado[coluna])
    
    return vetor_similaridades

In [234]:
matriz_similaridades = matriz_similarides(noticias_processado)

In [235]:
matriz_similaridades

array([[0.        , 0.30304576, 0.10101525, ..., 0.07412493, 0.08451543,
        0.        ],
       [0.30304576, 0.        , 0.14285714, ..., 0.        , 0.11952286,
        0.        ],
       [0.10101525, 0.14285714, 0.        , ..., 0.        , 0.11952286,
        0.        ],
       ...,
       [0.07412493, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08451543, 0.11952286, 0.11952286, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

13.3) - Depois que construímos a matriz de similaridades, faça o ranking das sentenças utilizando o método ``pagerank`` do pacote `networkx`. Este método irá retornar um score para cada sentença. Para utilizar este método, os dados devem representados em um grafo. Para isto converta a matriz de similaridades para um grafo de similaridades, utilizando o método `network.from_numpy_array(matrix)`.

<i>Mais informações Pagerank: </i>https://www.agenciamestre.com/marketing-digital/o-que-e-pagerank/

In [236]:
grafo = nx.from_numpy_array(matriz_similaridades)

In [237]:
scores = nx.pagerank(grafo)

In [238]:
sorted(scores.items(), key=operator.itemgetter(1))

[(16, 0.004021447721179649),
 (19, 0.004021447721179649),
 (15, 0.005655670158681687),
 (5, 0.007559632483027835),
 (28, 0.011320861876775489),
 (6, 0.01132219808546154),
 (36, 0.01245876816915345),
 (17, 0.012708394204105302),
 (24, 0.013217971142945685),
 (33, 0.013841617778291656),
 (38, 0.0146017509648837),
 (12, 0.015218666187634662),
 (14, 0.01613465308785664),
 (23, 0.016518064910325904),
 (30, 0.017089361220365713),
 (29, 0.017806214843770155),
 (31, 0.019035833922522026),
 (35, 0.02059076662993843),
 (3, 0.02094614009349831),
 (9, 0.024800621105511577),
 (27, 0.025031278859730773),
 (25, 0.026914108600851566),
 (37, 0.028719727389980774),
 (18, 0.030061246033376107),
 (2, 0.03034500646445102),
 (34, 0.030376390921891507),
 (20, 0.03668863971721915),
 (7, 0.036869708930105674),
 (11, 0.03733034100407843),
 (4, 0.038398763844942826),
 (26, 0.03872999251957474),
 (8, 0.0392727163458728),
 (13, 0.04030790516801762),
 (1, 0.0405190736215747),
 (10, 0.040656936621152974),
 (21, 0.04

13.4) Ordene os scores retornados pelo `pagerank` e imprima as <b> 5 </b> sentenças com maior score.

In [239]:
index_selecionados = (sorted(scores.items(), key=operator.itemgetter(1), reverse=True)[:5])

In [240]:
indices_rank = [i[0] for i in index_selecionados]

In [241]:
texto = ""
for i in indices_rank:
    texto += " " + noticia_sents[i]

In [242]:
texto

' No sábado (29), a Secretaria Estadual de Saúde e o Ministério da Saúde confirmaram o segundo caso de coronavírus no estado.  A Secretaria Estadual de Saúde de São Paulo divulgou nesta terça-feira (3) que caiu o número de pacientes com suspeita de coronavírus no estado. Após a confirmação de um caso, o governo estadual anunciou a criação de um centro de contingência para monitorar casos de coronavírus no estado. Segundo a metodologia da Secretaria Estadual de Saúde, para um caso ser considerado suspeito é necessário que o paciente tenha apresentado, além dos sintomas, histórico de viagem ou contato com caso suspeito. “Existem contactantes [do segundo caso], eles estão sendo monitorados, mas a gente não está divulgando mais números de contactantes por conta da privacidade das pessoas”, disse Paulo Menezes, coordenador do comitê de operações emergenciais (COE) do estado.'

<b> Atividade </b>

14) Analise o resultado da sumarização, construindo uma crítica sobre a qualidade e relevância da mesma.

O sumário ficou bem conciso e objetivo. O texto resumiu bem os fatos que ocorreram na época dos primeiros casos de COVID falando sobre os primeiros números de casos e os locais onde ocorreram. Achei interessante ter adicionado uma citação do coordenador do COE, Paulo Menezes. O texto parece ter sido escrito por uma humano.

<b> Atividade </b>

15) Faça o exercício 13 mais uma vez, agora substituindo a representação BoW pela TFIDF, usando o TDIDFVectorizer

In [271]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [272]:
def similaridade_TfIDF(s1, s2):
    palavras_unicas = (set(s1 + s2))
    model = TfidfVectorizer(binary=True)
    model.fit(palavras_unicas)
    vetor_s1 = model.transform([' '.join(s1)]).todense()
    vetor_s2 = model.transform([' '.join(s2)]).todense()    
    return cosine_similarity(vetor_s1, vetor_s2).reshape(-1)[0]

In [273]:
qtde_sents = len(noticia_sents)

In [274]:
vetor_similaridades = np.zeros((len(noticia_sents) , len(noticia_sents)))

In [275]:
similaridade(noticias_processado[0], noticias_processado[1])

0.3030457633656632

In [276]:
for linha in range(0, qtde_sents):
    for coluna in range(0, qtde_sents):
        if linha == coluna:
            continue
        vetor_similaridades[linha][coluna] = similaridade_TfIDF(noticias_processado[linha], noticias_processado[coluna])
        

In [278]:
def matriz_similarides_tfidf(sents):
    qtde_sents = len(noticia_sents)
    vetor_similaridades = np.zeros((len(noticia_sents) , len(noticia_sents)))
    for linha in range(0, qtde_sents):
        for coluna in range(0, qtde_sents):
            if linha == coluna:
                continue
            vetor_similaridades[linha][coluna] = similaridade_TfIDF(noticias_processado[linha], noticias_processado[coluna])
    
    return vetor_similaridades

In [279]:
matriz_similaridades = matriz_similarides_tfidf(noticias_processado)

In [280]:
matriz_similaridades

array([[0.        , 0.30304576, 0.10101525, ..., 0.07412493, 0.08451543,
        0.        ],
       [0.30304576, 0.        , 0.14285714, ..., 0.        , 0.11952286,
        0.        ],
       [0.10101525, 0.14285714, 0.        , ..., 0.        , 0.11952286,
        0.        ],
       ...,
       [0.07412493, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08451543, 0.11952286, 0.11952286, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [281]:
grafo = nx.from_numpy_array(matriz_similaridades)

In [282]:
scores = nx.pagerank(grafo)

In [283]:
sorted(scores.items(), key=operator.itemgetter(1))

[(16, 0.004021447721179649),
 (19, 0.004021447721179649),
 (15, 0.005655670158681687),
 (5, 0.007559632483027836),
 (28, 0.01132086187677549),
 (6, 0.011322198085461537),
 (36, 0.01245876816915345),
 (17, 0.012708394204105302),
 (24, 0.013217971142945685),
 (33, 0.01384161777829166),
 (38, 0.014601750964883704),
 (12, 0.015218666187634664),
 (14, 0.016134653087856644),
 (23, 0.016518064910325897),
 (30, 0.017089361220365713),
 (29, 0.017806214843770162),
 (31, 0.019035833922522026),
 (35, 0.020590766629938435),
 (3, 0.02094614009349831),
 (9, 0.024800621105511584),
 (27, 0.025031278859730777),
 (25, 0.02691410860085157),
 (37, 0.028719727389980777),
 (18, 0.030061246033376114),
 (2, 0.03034500646445103),
 (34, 0.030376390921891518),
 (20, 0.036688639717219154),
 (7, 0.036869708930105674),
 (11, 0.03733034100407844),
 (4, 0.038398763844942826),
 (26, 0.03872999251957474),
 (8, 0.03927271634587281),
 (13, 0.04030790516801762),
 (1, 0.04051907362157469),
 (10, 0.04065693662115298),
 (21, 

In [284]:
index_selecionados = (sorted(scores.items(), key=operator.itemgetter(1), reverse=True)[:5])

In [285]:
indices_rank = [i[0] for i in index_selecionados]

In [286]:
texto_tfidf = ""
for i in indices_rank:
    texto_tfidf += " " + noticia_sents[i]

In [287]:
texto_tfidf

' No sábado (29), a Secretaria Estadual de Saúde e o Ministério da Saúde confirmaram o segundo caso de coronavírus no estado.  A Secretaria Estadual de Saúde de São Paulo divulgou nesta terça-feira (3) que caiu o número de pacientes com suspeita de coronavírus no estado. Após a confirmação de um caso, o governo estadual anunciou a criação de um centro de contingência para monitorar casos de coronavírus no estado. Segundo a metodologia da Secretaria Estadual de Saúde, para um caso ser considerado suspeito é necessário que o paciente tenha apresentado, além dos sintomas, histórico de viagem ou contato com caso suspeito. “Existem contactantes [do segundo caso], eles estão sendo monitorados, mas a gente não está divulgando mais números de contactantes por conta da privacidade das pessoas”, disse Paulo Menezes, coordenador do comitê de operações emergenciais (COE) do estado.'

In [288]:
texto

' No sábado (29), a Secretaria Estadual de Saúde e o Ministério da Saúde confirmaram o segundo caso de coronavírus no estado.  A Secretaria Estadual de Saúde de São Paulo divulgou nesta terça-feira (3) que caiu o número de pacientes com suspeita de coronavírus no estado. Após a confirmação de um caso, o governo estadual anunciou a criação de um centro de contingência para monitorar casos de coronavírus no estado. Segundo a metodologia da Secretaria Estadual de Saúde, para um caso ser considerado suspeito é necessário que o paciente tenha apresentado, além dos sintomas, histórico de viagem ou contato com caso suspeito. “Existem contactantes [do segundo caso], eles estão sendo monitorados, mas a gente não está divulgando mais números de contactantes por conta da privacidade das pessoas”, disse Paulo Menezes, coordenador do comitê de operações emergenciais (COE) do estado.'

<b> Atividade </b>

16) Faça uma comparação dos resultados de sumarização obtidos pelas técnicas de Bag of Words e TFIDF, explicitando as diferenças observadas de forma concreta e exemplificando suas análises com frases dos textos.

O texto gerado automaticamente foi o mesmo para as duas abordagens. Possivelmente por conta do parâmetro "binary=True" informado em ambos os métodos.

<b> Desafio! (opcional, valendo pontos extras) </b>

Implemente e analise os resultados de uma solução que faça a sumarização dos textos utilizando Word Embeddings + WMD